<a href="https://colab.research.google.com/github/jvaaguiar/Img2Mesh/blob/master/Img2Mesh_Training_(Normal_Loss).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#[!] Training config (in/out)

In [0]:
from google.colab import drive
_ = drive.mount('/content/drive')

In [0]:
tarpath  = '/content/drive/\'My Drive\'/Img2Mesh/dataset/tars/'

In [0]:
# path to save checkpoints during the training
path_checkpoints_save = '/content/drive/\'My Drive\'/Img2Mesh/checkpoints/'
checkpoint_zip_name = 'checkpoint_normal.zip'

mapping_mesh = '/content/Img2Mesh/models/ico_162.obj'
out_vertices = 162 # number of outputs must match the mapping mesh
hidden_size = 4096

# Requirements

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
_ = !git clone https://github.com/jvaaguiar/Img2Mesh.git
import sys
sys.path.append('/content/Img2Mesh')

# Prepare Dataset

## Fetch data

In [0]:
datapath = "/content/Img2Mesh/data"
!mkdir {datapath}

tars = !ls -1 {tarpath}
for tar in tars:
  !tar -xf {tarpath + tar} -C {datapath}

In [0]:
paths = []
for t in tars:
  paths.append(datapath + '/' + t.replace('.tar.gz', '/train'))

In [0]:
from train_mesh import prepareTrainData
X, Y, faces = prepareTrainData(paths, ratio=1)
print(X.shape)

## Verify models sizes in Y

In [0]:
import matplotlib.pyplot as plt
import numpy as np
sizes = [y.shape[0] for y in Y]
sizes = np.array(sizes)
_ = plt.hist(sizes, bins=int(1e2))
plt.show()
_ = plt.hist(sizes[sizes < 1e4], bins=int(1e2))

## Downsample Y

In [0]:
from prepare_data import downsample
y, y_normals = downsample(Y, k=int(0.5e3), get_normals=True, faces=faces)

# Train

In [0]:
# save the model's weights at the end of each batch
drive_chks = path_checkpoints_save + checkpoint_zip_name
model_chks = '/content/Img2Mesh/checkpoints/'
!mkdir /content/Img2Mesh/checkpoints/

def download(path):
  _ = !zip -r {drive_chks} {path}

In [0]:
from train_mesh import saveCheckpoint
from train_mesh import runTraining, prepareNN


net, optim = prepareNN(out_verts = out_vertices,
                       hidden_size = hidden_size,
                       learning_rate=1e-4,
                       targ_obj_path = mapping_mesh,
                       norm_weight=0.9)

losses = runTraining(optim, X, y, y_normals,
                     batch_size=int(0.05*len(X)),
                     min_error=1e-3, min_step=1e-5,
                     max_repets=5,
                     num_epochs=20,
                     checkpoint_callback=saveCheckpoint(
                          path = '/content/Img2Mesh/checkpoints/',
                          download_callback=download))

In [0]:
# the trainning can be continued as many times as necessary running this cell
losses = runTraining(optim, X, y, y_normals,
                     batch_size=int(0.05*len(X)),
                     min_error=1e-3, min_step=1e-5,
                     max_repets=5,
                     num_epochs=20,
                     checkpoint_callback=saveCheckpoint(
                          path = '/content/Img2Mesh/checkpoints/check',
                          download_callback=download))

In [0]:
# test the network for an example from the training set
import matplotlib.pyplot as plt
id = 100
y_ = net(X[id:id+1])
fig, axs = plt.subplots(3, 2, sharex=True, sharey=True, figsize=(8,12))
axs[0, 0].set_xlim(0, 1), axs[0, 0].set_ylim(0, 1)

_ = axs[0, 0].scatter(y_[0][:,0],    y_[0][:,2],    c=y_[0][:,1])
_ = axs[0, 1].scatter(Y[id][:,0],    Y[id][:,2],    c=Y[id][:,1])
_ = axs[1, 0].scatter(y_[0][:,1],    y_[0][:,2],    c=y_[0][:,0])
_ = axs[1, 1].scatter(Y[id][:,1],    Y[id][:,2],    c=Y[id][:,0])
_ = axs[2, 0].scatter(y_[0][:,0],    y_[0][:,1],    c=y_[0][:,2])
_ = axs[2, 1].scatter(Y[id][:,0],    Y[id][:,1],    c=Y[id][:,2])